# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lreg").getOrCreate()

21/10/29 15:40:05 WARN Utils: Your hostname, xtian-pc resolves to a loopback address: 127.0.1.1; using 192.168.1.39 instead (on interface enp5s0)
21/10/29 15:40:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/29 15:40:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [13]:
from pyspark.ml.regression import LinearRegression

In [14]:
All_Data = spark.read.csv("cruise_ship_info.csv", 
                          inferSchema=True, 
                          header=True)

In [15]:
All_Data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [80]:
All_Data.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [37]:
Desc = All_Data.describe()
Desc.columns
Desc.select("summary", "Tonnage", "passengers", "length", "cabins", "crew").show()

+-------+------------------+-----------------+-----------------+------------------+-----------------+
|summary|           Tonnage|       passengers|           length|            cabins|             crew|
+-------+------------------+-----------------+-----------------+------------------+-----------------+
|  count|               158|              158|              158|               158|              158|
|   mean| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|7.794177215189873|
| stddev|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615|3.503486564627034|
|    min|             2.329|             0.66|             2.79|              0.33|             0.59|
|    max|             220.0|             54.0|            11.82|              27.0|             21.0|
+-------+------------------+-----------------+-----------------+------------------+-----------------+



## Dealing with the Cruise_line categorical variable
Ship Name is a useless arbitrary string, but the cruise_line itself may be useful. Let's make it into a categorical variable!

In [81]:
All_Data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [84]:
from pyspark.ml.feature import StringIndexer

In [91]:
Indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
All_Data = Indexer.fit(All_Data).transform(All_Data)
All_Data.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|       1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|       1.0|
|    Elation|   Carnival| 15

Arrange the data into ["features", "label"]

In [92]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [95]:
All_Data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [96]:
Asm = VectorAssembler(
    inputCols=[ "Tonnage",
                 "passengers",
                 "length",
                 "cabins",
                 "passenger_density",
                 "cruise_cat"],
      outputCol="features")

In [97]:
AsmOut = Asm.transform(All_Data)

In [98]:
Final_Data = AsmOut.select("features",'crew')

In [102]:
Train_Data,Test_Data = Final_Data.randomSplit([0.7,0.3])

In [103]:
Train_Data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               112|
|   mean| 7.797946428571431|
| stddev|3.6636965378465174|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [104]:
Test_Data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                46|
|   mean| 7.785000000000002|
| stddev|3.1168415280715047|
|    min|               0.6|
|    max|              13.6|
+-------+------------------+



In [105]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew')

In [106]:
# Fit the model to the data
lrModel = lr.fit(Train_Data)

21/10/29 16:04:43 WARN Instrumentation: [79518959] regParam is zero, which might cause numerical instability and overfitting.


In [107]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [0.017844221579568567,-0.16689719821814056,0.34430634809481075,0.8899063317439193,0.0013790518937221695,0.051501184090781645] Intercept: -1.3333491736051613


In [108]:
Test_Results = lrModel.evaluate(Test_Data)

In [109]:
Test_Results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.28588869775486947|
| -0.6111109142516932|
|-0.06445642234352045|
|  1.1228479949665893|
| 0.15380299413783094|
|  0.1154449204089607|
| 0.40110631244523853|
| 0.12864509798095458|
|  0.7054800813612037|
|  0.4362120535296121|
|-0.10884228203960156|
| 0.31111113450988803|
|0.019719513483795303|
|  1.0131006547504864|
| -1.1349640963541905|
| 0.12625605422541675|
|-0.20156445408499035|
| -0.6139077094339704|
| -0.6548913439643176|
| -0.2913233899993948|
+--------------------+
only showing top 20 rows



In [110]:
Unlabeled_Data = Test_Data.select('features')

In [111]:
Predictions = lrModel.transform(Unlabeled_Data)

In [112]:
Predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[2.329,0.94,2.96,...|0.3141113022451305|
|[5.35,1.58,4.4,0....|1.4911109142516932|
|[16.852,9.52,5.41...|3.0344564223435206|
|[19.093,8.0,5.37,...| 3.577152005033411|
|[25.0,7.76,6.22,3...| 3.696197005862169|
|[30.2769999999999...|3.8845550795910393|
|[30.2769999999999...|3.3288936875547614|
|[34.25,10.52,6.15...| 4.571354902019046|
|[35.143,12.5,6.69...| 4.644519918638796|
|[38.0,7.49,6.74,3...|4.1637879464703875|
|[38.0,10.56,5.67,...|4.4888422820396015|
|[42.0,14.8,7.13,7...| 6.488888865490112|
|[42.0,15.04,7.08,...|6.2802804865162045|
|[48.563,20.2,6.92...| 5.696899345249514|
|[50.76,17.48,7.54...|  7.27496409635419|
|[52.926,13.02,7.1...| 6.043743945774583|
|[53.049,13.44,7.2...|  6.20156445408499|
|[53.872,14.94,7.9...|6.7339077094339705|
|[53.872,14.94,7.9...| 7.014891343964318|
|[55.451,12.64,7.1...| 5.861323389999395|
+--------------------+------------

In [119]:
print("RMSE: {}".format(Test_Results.rootMeanSquaredError))
print("MSE: {}".format(Test_Results.meanSquaredError))
print("R2: {}".format(Test_Results.r2))

RMSE: 0.7605157388401846
MSE: 0.578384189023632
R2: 0.9391399525070604


In [120]:
# Check the data a little closer
from pyspark.sql.functions import corr

In [122]:
All_Data.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [123]:
All_Data.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

